This notebook is developed using the `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel on an `ml.t3.medium` instance.

In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter07'

In [ ]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

In [ ]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

csv_test_dir_prefix = 'imdb_data/test'
csv_test_filename = 'test.csv'
csv_test_dir = os.path.join(os.getcwd(), csv_test_dir_prefix)
os.makedirs(csv_test_dir, exist_ok=True)

np.savetxt(os.path.join(csv_test_dir, csv_test_filename), 
           np.array(x_test, dtype=np.int32), fmt='%d', delimiter=",")

test_data_s3prefix = f'{prefix}/data/csv_test'
test_data_s3 = sess.upload_data(path=csv_test_dir, 
                                key_prefix=test_data_s3prefix)
print(test_data_s3)

In [ ]:
from sagemaker.tensorflow import TensorFlow

# insert the training job name from chapter05/02-tensorflow_sentiment_analysis.ipynb
training_job_name= '<your-training-job-name>' # e.g. imdb-tf-2022-02-18-05-31-26

estimator = TensorFlow.attach(training_job_name)

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
from time import gmtime, strftime

experiment_name = 'imdb-sentiment-analysis'

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-tf-bt-{exp_datetime}'

# Creating a new trial for the experiment
exp_trial = Trial.load(trial_name=training_job_name)

experiment_config={
    'ExperimentName': experiment_name,
    'TrialName': exp_trial.trial_name,
    'TrialComponentDisplayName': 'Inference-BatchTransform'}

In [ ]:
exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-tf-bt-{exp_datetime}'

s3_output_location = f's3://{bucket}/{prefix}/{jobname}'

transformer = estimator.transformer(instance_count=1, 
                                    instance_type='ml.c5.xlarge',
                                    max_payload = 2, # MB
                                    accept = 'application/jsonlines',
                                    output_path = s3_output_location,
                                    assemble_with = 'Line')

transformer.transform(test_data_s3, 
                      content_type='text/csv', 
                      split_type = 'Line', 
                      job_name = jobname,
                      experiment_config = experiment_config)
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)

In [ ]:
output = transformer.output_path
output_prefix = 'imdb_data/test_output'
!mkdir -p {output_prefix}
!aws s3 cp --recursive {output} {output_prefix}
!head {output_prefix}/{csv_test_filename}.out

In [ ]:
results=[]
with open(f'{output_prefix}/{csv_test_filename}.out', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line)
        json_output = json.loads(line)
        result = [float('%.3f'%(item)) for sublist in json_output['predictions'] 
                                       for item in sublist]
        results += result

print(results)

In [ ]:
def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

In [ ]:
import re

regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()

In [ ]:
data_index=199
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') 
                                 for i in x_test[data_index]])
regex.sub('', first_decoded_review)

In [ ]:
print(f'Labeled sentiment for this review is {get_sentiment(y_test[data_index])}')
print(f'Predicted sentiment is {get_sentiment(results[data_index])}')